In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/cse158_assignment2'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/cse158_assignment2


In [ ]:
import gzip
import json
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import nltk
nltk.download("popular")
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [ ]:
f = gzip.open("reviews_Beauty_5.json.gz")
dataset = []
for l in f:
  dataset.append(json.loads(l))
print(len(dataset))


198502


In [ ]:
dataset = [d for d in dataset if d["overall"] != 3]

print(len(dataset))
print(dataset[0])

176254
{'reviewerID': 'A1YJEY40YUW4SE', 'asin': '7806397051', 'reviewerName': 'Andrea', 'helpful': [3, 4], 'reviewText': 'Very oily and creamy. Not at all what I expected... ordered this to try to highlight and contour and it just looked awful!!! Plus, took FOREVER to arrive.', 'overall': 1.0, 'summary': "Don't waste your money", 'unixReviewTime': 1391040000, 'reviewTime': '01 30, 2014'}


In [ ]:
for d in dataset:
  if d["overall"] >3:
    d["positive_rating"] = 1
  else:
    d["positive_rating"] = 0

dataset_train = dataset[:140000]
dataset_test = dataset[140000:]

print(dataset[0])

{'reviewerID': 'A1YJEY40YUW4SE', 'asin': '7806397051', 'reviewerName': 'Andrea', 'helpful': [3, 4], 'reviewText': 'Very oily and creamy. Not at all what I expected... ordered this to try to highlight and contour and it just looked awful!!! Plus, took FOREVER to arrive.', 'overall': 1.0, 'summary': "Don't waste your money", 'unixReviewTime': 1391040000, 'reviewTime': '01 30, 2014', 'positive_rating': 0}


In [ ]:
print(len(dataset))
print(sum([d["positive_rating"] for d in dataset]))

176254
154272


In [ ]:


wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset_train:
  r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
  for w in r.split():
    # if w in stop_words:
    #   continue
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
print(len(counts))
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:2000]]
print(words[:10])

wordId = dict(zip(words,range(len(words))))
wordSet = set(words)

def feature(datnum):
  r = ''.join([c for c in datnum['reviewText'].lower() if not c in punctuation])
  f = [0]*len(words)
  counter = 0
  for w in r.split():
    if w in wordSet:
      f[wordId[w]] += 1
      counter+=1
  return f + [1]

x_train = [feature(d) for d in dataset_train]
y_train = [d["positive_rating"] for d in dataset_train]


mod = linear_model.LogisticRegression(C=1)
mod.fit(x_train,y_train)

x_test = [feature(d) for d in dataset_test]
y_test = [d["positive_rating"] for d in dataset_test]

pred = mod.predict(x_test)
print(len(pred))

print(pred[:10])
print(y_test[:10])

def accuracy(predictions, y):
  TP = sum([(p == False and l == False) for (p,l) in zip(predictions,y)])
  FP = sum([(p == False and l == True) for (p,l) in zip(predictions,y)])
  TN = sum([(p == True and l == True) for (p,l) in zip(predictions,y)])
  FN = sum([(p == True and l == False) for (p,l) in zip(predictions,y)])
  return (TP+TN)/(TP+FP+TN+FN)

print(accuracy(pred, y_test))
print(accuracy([1]*len(y_test),y_test))

142131
['i', 'the', 'and', 'it', 'a', 'to', 'this', 'my', 'is', 'of']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


36254
[1 1 1 0 1 1 1 1 1 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0.9381861311855244
0.9059414133612843


In [ ]:
#################################################### change cut off to 4 ##########################################################

In [ ]:
f = gzip.open("reviews_Beauty_5.json.gz")
dataset = []
for l in f:
  dataset.append(json.loads(l))
print(len(dataset))
dataset = [d for d in dataset if d["overall"] != 4]

print(len(dataset))
print(dataset[0])


for d in dataset:
  if d["overall"] >4:
    d["positive_rating"] = 1
  else:
    d["positive_rating"] = 0

dataset_train = dataset[:130000]
dataset_test = dataset[130000:]

print(len(dataset))
print(sum([d["positive_rating"] for d in dataset]))


198502
158761
{'reviewerID': 'A1YJEY40YUW4SE', 'asin': '7806397051', 'reviewerName': 'Andrea', 'helpful': [3, 4], 'reviewText': 'Very oily and creamy. Not at all what I expected... ordered this to try to highlight and contour and it just looked awful!!! Plus, took FOREVER to arrive.', 'overall': 1.0, 'summary': "Don't waste your money", 'unixReviewTime': 1391040000, 'reviewTime': '01 30, 2014'}
158761
114531


In [ ]:




wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset_train:
  r = ''.join([c for c in d['reviewText'].lower() if not c in punctuation])
  for w in r.split():
    # if w in stop_words:
    #   continue
    wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
print(len(counts))
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:2000]]
print(words[:10])

wordId = dict(zip(words,range(len(words))))
wordSet = set(words)

def feature(datnum):
  r = ''.join([c for c in datnum['reviewText'].lower() if not c in punctuation])
  f = [0]*len(words)
  counter = 0
  for w in r.split():
    if w in wordSet:
      f[wordId[w]] += 1
      counter+=1
  return f + [1]

x_train = [feature(d) for d in dataset_train]
y_train = [d["positive_rating"] for d in dataset_train]


mod = linear_model.LogisticRegression(C=1)
mod.fit(x_train,y_train)

x_test = [feature(d) for d in dataset_test]
y_test = [d["positive_rating"] for d in dataset_test]

pred = mod.predict(x_test)
print(len(pred))

print(pred[:10])
print(y_test[:10])

def accuracy(predictions, y):
  TP = sum([(p == False and l == False) for (p,l) in zip(predictions,y)])
  FP = sum([(p == False and l == True) for (p,l) in zip(predictions,y)])
  TN = sum([(p == True and l == True) for (p,l) in zip(predictions,y)])
  FN = sum([(p == True and l == False) for (p,l) in zip(predictions,y)])
  return (TP+TN)/(TP+FP+TN+FN)

print(accuracy(pred, y_test))
print(accuracy([1]*len(y_test),y_test))

133606
['i', 'the', 'and', 'it', 'a', 'to', 'this', 'my', 'is', 'of']


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


28761
[0 1 1 1 0 1 1 1 1 1]
[0, 1, 1, 1, 0, 1, 1, 0, 1, 1]
0.8800111261778102
0.7375265115955635
